In [204]:
%matplotlib notebook

In [205]:
from microphone import record_audio
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from IPython.display import Audio
import librosa

from numba import njit
import numpy as np
import pickle
import time

from music21 import *
import pygame as pg

from sklearn.cluster import KMeans

In [206]:
us = environment.UserSettings()
us.getSettingsPath()
environment.set('musescoreDirectPNGPath', "/usr/bin/musescore")

Getting User Input

In [313]:
def micRecord(time=10):
    frames, rate = record_audio(time)
    return np.hstack([np.frombuffer(i, np.int16) for i in frames]), rate
def getFile(path):
    recorded_audio, sampling_rate = librosa.load(path, 
                                                 sr=44100, 
                                                 mono=True,
                                                duration=30)
    return recorded_audio, sampling_rate
def pressure(times, *, amp, freq):
    return amp * np.sin(2 * np.pi * freq * times)
def pureTone(freq, amp=0.06, dur=3, rate=44100):
    n_samples = int(dur * rate) + 1
    times = np.arange(n_samples) / rate  # seconds
    return pressure(times, amp=amp, freq=freq), rate  # Pascals

In [208]:
def userinput():
    while True:
        audioType = input("u for Upload, r for Record: ")
        if audioType == 'u':
            path = input("Enter path to file: ")
            samples, rate = getFile(path)
            break
        elif audioType == 'r':
            samples, rate = micRecord()
            break
        print("Invalid input. Try again.")
    # print(rate)
    
    return samples, rate

Processing

In [209]:
def getspec(samples, rate):
    spectrogram, freqs, times = mlab.specgram(
		samples,
		NFFT=4096,
		Fs=rate,
		window=mlab.window_hanning,
		noverlap=int(4096 / 2)
	)
    return spectrogram, freqs, times

In [210]:
def distinctFreqs(freq1, freq2, a=1.059):
    larger_freq = max(freq1, freq2)
    smaller_freq = min(freq1, freq2)
    if smaller_freq == 0:
        smaller_freq = 1e-100
    if larger_freq/smaller_freq >= a:
        return True
    else:
        return False

In [311]:
def convert(freqTimes):
    #returns a list of lists [freq, start, last seen instance, duration]
    freqDurations = []
    
    freqTimes = list(freqTimes.values())
    freqTimes.append(0)
    # key and value for the original dictionary
    time=0
    curlength = 0
    curnote = 0
    
    concat = True
    while time<len(freqTimes):
        if distinctFreqs(freqTimes[time], curnote):
            if curnote!=0 and curlength>2:
                #if concat:
                freqDurations.append([curnote, time-curlength, curlength])
                #else:
                #    freqDurations[-1][2]+=curlength
                #    concat=True
                curlength = 0
                curnote = freqTimes[time]
            elif curnote==0 and curlength>2:
                curlength = 0
                curnote = freqTimes[time]
            elif not distinctFreqs(freqTimes[time], freqDurations[-1][0]):
                concat = False
                curnote = freqDurations[-1][0]
            else:
                curlength = 0
                curnote = freqTimes[time]
        else:
            curlength+=1
        time+=1
    return freqDurations

In [212]:
def durations(num_clusters):
    #returns list of durations
    quarter_index = int(num_clusters/2)
    
    durationMappingKey = [0.166, 0.5, 1, 2, 4]
    
    duration_list = []
    start = 2 - quarter_index
    for i in range(start, start + num_clusters):
        duration_list.append(durationMappingKey[i])
        return duration_list

In [344]:
conversions = {
    pitch.Accidental("sharp"):pitch.Accidental("sharp"),
    pitch.Accidental("half-sharp"):None,
    pitch.Accidental("one-and-a-half-sharp"):pitch.Accidental("sharp"),
    None: None
}

def notes(freqDurations, bpm, timestep):
    #return list of notes
    #normFreqs = music21.audioSearch.detectPitchFrequencies([freq[0] for freq in freqDurs])
    notes = []
    duration_list = np.array([0.166, 0.5, 1, 2, 4])
    quarterlength = 60/bpm
    absdurs = duration_list * quarterlength
    
    for freqIndex in range(len(freqDurations)):
        p = pitch.Pitch()
        p.frequency = freqDurations[freqIndex][0]
        #if p.accidental==pitch.Accidental()
        #    p.accidental=pitch.Accidental('double-flat')
        p.accidental = conversions[p.accidental]
        n = note.Note(pitchName = p.name)
        
        d = duration.Duration()
        notelength = freqDurations[freqIndex][2] * timestep
        closest = np.argmin(np.abs(absdurs - notelength))
        d.quarterLength = duration_list[closest]
        print(d.quarterLength)
        n.duration = d
        
        notes.append(n)
    #print(normFreqs)
    return notes

In [214]:
def streamNotes(notes):
    s = stream.Stream()
    for n in notes:
        s.append(n)
#     mf = midi.translate.streamToMidiFile(s)
    s.show()
    #sp = midi.realtime.StreamPlayer(s, playForMilliseconds = 5000)
    #sp.play()
#     return mf

In [324]:
def sheetMusic(peak_dict, bpm, timestep):
    freqDurs = convert(peak_dict)
    print(freqDurs)
        
    quarterlength = 60/bpm
    
    notesList = notes(freqDurs, bpm, timestep)
    streamNotes(notesList)

In [216]:
def find_min_amp(maxamps, amp_threshold):
    ind = round(len(maxamps) * amp_threshold)
    cutoff_log_amplitude = np.partition(maxamps, ind)[ind]
    return cutoff_log_amplitude

In [346]:
#music, sampling_rate = pureTone(103.83)
music, sampling_rate = userinput()
spec, freq, time = getspec(music, sampling_rate)

maxamps = np.log(np.max(spec,axis=0))
maxfreqs = np.argmax(spec, axis=0)
maxfreqs = freq[maxfreqs]

timestep = time[1]-time[0]
print(timestep)
thresh = find_min_amp(maxamps,0.2)
comps = maxamps>thresh
zeroedfreqs = comps*maxfreqs

#print(zeroedfreqs)

timefreq = [(i,j) for i,j in enumerate(zeroedfreqs)]
timefreq = dict(timefreq)

sheetMusic(timefreq, bpm = 120, timestep=timestep)

u for Upload, r for Record: u
Enter path to file: twinkle.mp3


/home/alanxw/miniconda3/envs/week1/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


0.046439909297052155
[[527.5634765625, 8, 11], [527.5634765625, 21, 15], [785.9619140625, 38, 9], [785.9619140625, 53, 5], [785.9619140625, 60, 5], [882.861328125, 68, 12], [882.861328125, 82, 13], [796.728515625, 97, 13], [699.8291015625, 127, 10], [699.8291015625, 142, 13], [656.7626953125, 158, 25], [602.9296875, 186, 30], [527.5634765625, 217, 13], [796.728515625, 246, 8], [796.728515625, 256, 3], [796.728515625, 261, 6], [699.8291015625, 276, 12], [699.8291015625, 290, 8], [699.8291015625, 300, 5], [656.7626953125, 306, 28], [581.396484375, 335, 17], [592.1630859375, 361, 3], [785.9619140625, 365, 6], [785.9619140625, 376, 11], [699.8291015625, 394, 12], [699.8291015625, 408, 13], [656.7626953125, 425, 11], [656.7626953125, 438, 15], [592.1630859375, 454, 9], [592.1630859375, 468, 3], [527.5634765625, 484, 28], [785.9619140625, 513, 9], [796.728515625, 528, 9], [882.861328125, 543, 13], [882.861328125, 558, 13], [785.9619140625, 573, 9], [699.8291015625, 602, 12], [699.8291015625,

MusicXMLExportException: In part (None), measure (4): Cannot convert "2048th" duration to MusicXML (too short).